In [2]:
import random

def initialize_board():
    """Initialize the Othello board with the starting position."""
    board = [[' ' for _ in range(8)] for _ in range(8)]
    board[3][3], board[4][4] = 'W', 'W'
    board[3][4], board[4][3] = 'B', 'B'
    return board

###############################################################################################################
BOLD = "\033[1m"
RESET = "\033[0m"

# Cell representations
GREEN_BG = "\033[42m   \033[0m"              # Default green square
BLACK_PIECE = "\033[40m   \033[0m"            # Black piece
WHITE_PIECE = "\033[47m   \033[0m"            # White piece
VALID_MOVE_MARK = "\033[42m\033[1m\033[31m # \033[0m"  # Green background with bold red 

def get_square_display(cell, is_valid_move):
    """Return the display for each square, marking valid moves if applicable."""
    if cell == 'B':
        return BLACK_PIECE
    elif cell == 'W':
        return WHITE_PIECE
    elif is_valid_move:
        return VALID_MOVE_MARK 
    else:
        return GREEN_BG 

def print_board(board, valid_moves):
    """Print the Othello board with valid moves highlighted and dynamic color updates."""
    # Convert valid moves to a set for quick lookup
    valid_move_positions = set(valid_moves)

    # Print bold column headers
    print("    " + "   ".join(f"{BOLD}{i + 1}{RESET}" for i in range(8)))

    # Top border
    print("  ┌" + "───┬" * 7 + "───┐")

    for i, row in enumerate(board):
        # Print each row with bold row labels (a-h) and formatted cells
        row_display = [get_square_display(cell, (i, j) in valid_move_positions) for j, cell in enumerate(row)]
        print(f"{BOLD}{chr(97 + i)}{RESET} │" + "│".join(row_display) + "│")

        # Print row separators or bottom border
        if i < 7:
            print("  ├" + "───┼" * 7 + "───┤")
        else:
            print("  └" + "───┴" * 7 + "───┘")

    # Print the scores with bold styling
    black_score = sum(row.count('B') for row in board)
    white_score = sum(row.count('W') for row in board)
    print(f"\n{BOLD}Score -> Black: ⚫ {black_score}, White: ⚪ {white_score}{RESET}\n")

    
##############################################################################################################
def get_valid_moves(board, player):
    """Returns a list of valid moves for the given player."""
    return [(row, col) for row in range(8) for col in range(8) if is_valid_move(board, row, col, player)]


def is_valid_move(board, row, col, player):
    """Check if placing a piece at (row, col) is a valid move."""
    if board[row][col] != ' ':
        return False

    opponent = 'B' if player == 'W' else 'W'
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]

    for dr, dc in directions:
        r, c = row + dr, col + dc
        found_opponent = False

        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            found_opponent = True
            r += dr
            c += dc

        if found_opponent and 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            return True  # Valid move

    return False

def execute_move(board, row, col, player):
    """Place a piece and flip opponent's pieces accordingly."""
    opponent = 'B' if player == 'W' else 'W'
    board[row][col] = player  # Place piece

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (1, 1), (-1, 1), (1, -1)]

    for dr, dc in directions:
        r, c = row + dr, col + dc
        pieces_to_flip = []

        while 0 <= r < 8 and 0 <= c < 8 and board[r][c] == opponent:
            pieces_to_flip.append((r, c))
            r += dr
            c += dc

        if 0 <= r < 8 and 0 <= c < 8 and board[r][c] == player:
            for flip_r, flip_c in pieces_to_flip:
                board[flip_r][flip_c] = player  # Flip pieces

##############################################################################################################

POSITION_WEIGHTS = [
    [100, -20, 30, 10, 10, 30, -20, 100],
    [-20, -30, -2, -2, -2, -2, -30, -20],
    [ 30,  -2, -1, -1, -1, -1,  -2,  30],
    [ 10,  -2, -1,  0,  0, -1,  -2,  10],
    [ 10,  -2, -1,  0,  0, -1,  -2,  10],
    [ 30,  -2, -1, -1, -1, -1,  -2,  30],
    [-20, -30, -2, -2, -2, -2, -30, -20],
    [100, -20, 30, 10, 10, 30, -20, 100]
]


def evaluate_board(board, ai_color):
    """Evaluates the board using weighted positions."""
    opponent_color = 'B' if ai_color == 'W' else 'W'
    ai_score = sum(POSITION_WEIGHTS[row][col] for row in range(8) for col in range(8) if board[row][col] == ai_color)
    opponent_score = sum(POSITION_WEIGHTS[row][col] for row in range(8) for col in range(8) if board[row][col] == opponent_color)
    return ai_score - opponent_score  # AI wants a higher score


############################################################################################################

def ai_choose_move(board, ai_color):
    """AI chooses the best move using Minimax with Alpha-Beta Pruning."""
    depth = get_dynamic_depth(board)  # Adjust depth based on game stage
    print(f"🔍 AI Searching at Depth: {depth}")

    move_scores = {}
    alpha, beta = float('-inf'), float('inf')

    for move in get_valid_moves(board, ai_color):
        temp_board = [row[:] for row in board]
        execute_move(temp_board, move[0], move[1], ai_color)
        eval_score, _ = minimax(temp_board, depth - 1, alpha, beta, False, ai_color)
        move_scores[move] = eval_score

    best_move = max(move_scores, key=move_scores.get, default=(-1, -1))

    print(f"📌 AI at Depth {depth} - Move Scores: {move_scores}")
    print(f"🎯 AI Chooses Move: {best_move} with Score {move_scores.get(best_move, 0)}\n")

    return best_move

##########################################################################################################
def get_player_color():
    """Prompt the user to choose their color (Black or White)."""
    while True:
        player_color = input("Do you want to play as Black (B) or White (W)? ").strip().lower()
        if player_color in ['b', 'w']:
            return 'B' if player_color == 'b' else 'W'
        else:
            print("Invalid entry. Please enter 'B' for Black or 'W' for White.")

##########################################################################################################

def get_dynamic_depth(board):
    """Adjust search depth based on game progress."""
    empty_spaces = sum(row.count(' ') for row in board)
    return 3 if empty_spaces > 50 else 5 if empty_spaces > 30 else 7
    
##########################################################################################################

def get_time_limit():
    """Prompt the user to set a time limit for the AI's decision-making."""
    while True:
        try:
            time_limit = float(input("Enter the time limit for AI's move (seconds): ").strip())
            if time_limit > 0:
                return time_limit
            else:
                print("Time limit must be greater than 0.")
        except ValueError:
            print("Invalid input. Please enter a positive number.")
import time

def get_time_limited_depth(board, ai_color, max_depth=8, time_limit=1):
    """Determine the maximum depth that can be searched within the time limit using iterative deepening."""
    start_time = time.time()
    best_move = None
    last_successful_depth = 1  # Track the deepest depth that worked

    for depth in range(1, max_depth + 1):  # Start from depth 1 and go deeper
        try:
            _, best_move = minimax(board, depth, float('-inf'), float('inf'), True, ai_color, start_time, time_limit)
            last_successful_depth = depth  # Update successful depth
        except TimeoutError:
            break  # Stop searching when time runs out

    return last_successful_depth  # Return the deepest completed depth

##########################################################################################################
def get_dynamic_depth(board):
    """Adjust search depth based on game progress."""
    empty_spaces = sum(row.count(' ') for row in board)
    return 3 if empty_spaces > 50 else 5 if empty_spaces > 30 else 7
    
# Ask the user to select the depth strategy
def select_depth_strategy():
    print("Select AI Depth Strategy:")
    print("1. Dynamic Depth (Based on Board State)")
    print("2. Time-Limited Depth")
    while True:
        try:
            choice = int(input("Enter your choice (1 or 2): "))
            if choice in [1, 2]:
                return choice
            else:
                print("Invalid choice. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter a number (1 or 2).")

##########################################################################################################

def format_move_to_xy(row, col):
    """Convert row and column indices to XY format (e.g., d3)."""
    return f"{chr(97 + row)}{col + 1}"

def parse_xy_move(move_input):
    """Convert XY formatted move (e.g., d3) to row and column indices."""
    if len(move_input) == 2 and move_input[0] in 'abcdefgh' and move_input[1].isdigit():
        row = ord(move_input[0]) - 97  # Convert 'a'-'h' to 0-7
        col = int(move_input[1]) - 1   # Convert '1'-'8' to 0-7
        return row, col
    else:
        raise ValueError("Invalid move format")

#########################################################################################################

def minimax(board, depth, alpha, beta, maximizing_player, ai_color, start_time=None, time_limit=2, is_root=True):
    """Minimax algorithm with Alpha-Beta Pruning and optional time-limiting."""
    if start_time is None:
        start_time = time.time()
        
    # ✅ Return fallback value instead of crashing on timeout
    if time.time() - start_time > time_limit:
        return evaluate_board(board, ai_color), None  # Return current board evaluation


    opponent_color = 'B' if ai_color == 'W' else 'W'
    valid_moves = get_valid_moves(board, ai_color if maximizing_player else opponent_color)

    # Base case
    if depth == 0 or not valid_moves:
        return evaluate_board(board, ai_color), None

    move_scores = {}

    if maximizing_player:
        max_eval = float('-inf')
        best_move = None
        for move in valid_moves:
            temp_board = [row[:] for row in board]
            execute_move(temp_board, move[0], move[1], ai_color)
            eval_score, _ = minimax(temp_board, depth - 1, alpha, beta, False, ai_color, start_time, time_limit, False)

            move_scores[move] = eval_score

            if eval_score > max_eval:
                max_eval = eval_score
                best_move = move
            alpha = max(alpha, eval_score)
            if beta <= alpha:
                break  # Alpha-Beta pruning

        # ✅ **Print final decision at the root level only**
        #if is_root:
            #print(f"\n🔎 AI Depth {depth} Final Choices: {move_scores}")
            #print(f"🎯 AI Chose: {best_move} with Score {max_eval} (Expected Max: {max(move_scores.values())})\n")

        return max_eval, best_move

    else:  # Minimizing Player
        min_eval = float('inf')
        best_move = None
        for move in valid_moves:
            temp_board = [row[:] for row in board]
            execute_move(temp_board, move[0], move[1], opponent_color)
            eval_score, _ = minimax(temp_board, depth - 1, alpha, beta, True, ai_color, start_time, time_limit, False)

            move_scores[move] = eval_score

            if eval_score < min_eval:
                min_eval = eval_score
                best_move = move
            beta = min(beta, eval_score)
            if beta <= alpha:
                break  # Alpha-Beta pruning

        return min_eval, best_move
##################################################################################################################
def game_loop():
    """Main function to run the Othello game between a human and AI."""

    # Initialize the board
    board = initialize_board()

    player_color = get_player_color()
    computer_color = 'B' if player_color == 'W' else 'W'

    # Ask the user to select a depth strategy
    depth_strategy = select_depth_strategy()

    # Ask the user to provide a time limit for AI moves
    time_limit = get_time_limit()

    # Start with Black
    current_player = 'B'
    consecutive_passes = 0  # Track consecutive passes

    while True:
        valid_moves = get_valid_moves(board, current_player)  # Define valid moves first
        print_board(board, valid_moves)                      # Now print the board with valid moves

        # No valid moves? Pass turn
        if not valid_moves:
            print(f"{current_player} has no valid moves and must pass.")
            consecutive_passes += 1
            if consecutive_passes == 2:  # If both players pass consecutively, end game
                break
            current_player = 'B' if current_player == 'W' else 'W'
            continue
        else:
            consecutive_passes = 0  # Reset pass counter when a move is made

        # Human Player's Turn
        if current_player == player_color:
            print(f"{current_player}'s turn (Your turn):")
            formatted_moves = [format_move_to_xy(r, c) for r, c in valid_moves]
            print(f"Valid moves: {formatted_moves}")

            while True:
                try:
                    move_input = input("Enter your move (e.g., g4): ").strip().lower()
                    row, col = parse_xy_move(move_input)
                    if (row, col) in valid_moves:
                        execute_move(board, row, col, current_player)
                        break
                    else:
                        print("Invalid move. Try again.")
                except (ValueError, IndexError):
                    print("Invalid input. Enter a move in the format 'g4'.")

        # AI Turn
        else:
            print(f"{current_player}'s turn (Computer's turn):")

            if depth_strategy == 1:  # Dynamic Depth
                depth = get_dynamic_depth(board)
            else:  # Time-Limited Depth
                depth = get_time_limited_depth(board, computer_color, time_limit=time_limit)

            _, best_move = minimax(board, depth, float('-inf'), float('inf'), True, computer_color, time_limit=time_limit)
            if best_move:
                formatted_move = format_move_to_xy(best_move[0], best_move[1])
                execute_move(board, best_move[0], best_move[1], current_player)
                print(f"Computer chose: {formatted_move}")
            else:
                print(f"{current_player} has no valid moves and must pass.")

        # Switch players
        current_player = 'B' if current_player == 'W' else 'W'

    # Game Over - Print final board and scores
    valid_moves = []  # No valid moves at game end
    print_board(board, valid_moves)
    print("Game over!")
    black_score = sum(row.count('B') for row in board)
    white_score = sum(row.count('W') for row in board)
    print(f"Final Score - Black: {black_score}, White: {white_score}")

    if black_score > white_score:
        print("Black wins!")
    elif white_score > black_score:
        print("White wins!")
    else:
        print("It's a draw!")

#####################################################################################################


game_loop()


Do you want to play as Black (B) or White (W)?  B


Select AI Depth Strategy:
1. Dynamic Depth (Based on Board State)
2. Time-Limited Depth


Enter your choice (1 or 2):  1
Enter the time limit for AI's move (seconds):  1


    1   2   3   4   5   6   7   8
  ┌───┬───┬───┬───┬───┬───┬───┬───┐
a │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
b │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
c │   │   │   │ # │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
d │   │   │ # │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
e │   │   │   │   │   │ # │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
f │   │   │   │   │ # │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
g │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
h │   │   │   │   │   │   │   │   │
  └───┴───┴───┴───┴───┴───┴───┴───┘

Score -> Black: ⚫ 2, White: ⚪ 2

B's turn (Your turn):
Valid moves: ['c4', 'd3', 'e6', 'f5']


Enter your move (e.g., g4):  c5


...

Enter your move (e.g., g4):  f5


    1   2   3   4   5   6   7   8
  ┌───┬───┬───┬───┬───┬───┬───┬───┐
a │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
b │   │   │   │ # │   │ # │ # │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
c │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
d │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
e │   │   │   │   │   │ # │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
f │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
g │   │   │   │   │   │ # │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
h │   │   │   │   │   │   │   │   │
  └───┴───┴───┴───┴───┴───┴───┴───┘

Score -> Black: ⚫ 7, White: ⚪ 2

W's turn (Computer's turn):
Computer chose: e6
    1   2   3   4   5   6   7   8
  ┌───┬───┬───┬───┬───┬───┬───┬───┐
a │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
b │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
c │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼

Enter your move (e.g., g4):  f4


    1   2   3   4   5   6   7   8
  ┌───┬───┬───┬───┬───┬───┬───┬───┐
a │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
b │   │   │ # │   │ # │ # │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
c │   │   │ # │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
d │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
e │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
f │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
g │   │   │ # │ # │ # │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
h │   │   │   │   │   │   │   │   │
  └───┴───┴───┴───┴───┴───┴───┴───┘

Score -> Black: ⚫ 8, White: ⚪ 3

W's turn (Computer's turn):
Computer chose: c3
    1   2   3   4   5   6   7   8
  ┌───┬───┬───┬───┬───┬───┬───┬───┐
a │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
b │   │   │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼───┼───┼───┤
c │   │ # │   │   │   │   │   │   │
  ├───┼───┼───┼───┼───┼

KeyboardInterrupt: Interrupted by user